# Imports

In [85]:
# Third-Party Imports
import nltk
import torch
import requests as req
import openai
from dotenv import load_dotenv
load_dotenv()

# Standard Library Imports
import os
import json
from math import inf
from string import punctuation
import sys

# Local Imports
from queries import get_text_cli
from get_documents import search

# Utility Functions

In [2]:
def get_text():
    term = get_text_cli('Enter a search term')
    return search(term)

def load_docs(dirname='corpus'):
    corpus = dict()
    main_path = os.path.join(os.path.dirname('__file__'), dirname)

    for file in os.listdir(main_path):
        with open(os.path.join(main_path, file), 'r') as f:
            corpus[file] = f.read()
    
    return corpus

def chunk_text(text, chunk_len):
    chunks = []
    current_chunk = ""
    sents = nltk.sent_tokenize(text)
    
    for sent in sents:
        if len(nltk.word_tokenize(current_chunk + f" {sent}")) >= chunk_len:
            chunks.append(current_chunk)
            current_chunk = ""
        else:
            current_chunk += f" {sent}"
    
    chunks.append(current_chunk)
    
    return chunks

# Simple Extractive Text Summarisation

In [115]:
def word_tokenize(text):
    banned = list(punctuation) + nltk.corpus.stopwords.words("english")
    return [w for w in nltk.word_tokenize(text) if w not in banned]

def SE_summary(text, summary_size=0.5):
    # Create word and sentence tokens
    words = word_tokenize(text)
    word_set = set(words) # set of all unique words in word tokens
    sents = nltk.sent_tokenize(text)
    
    # Initialise frequency table for word tokens
    w_freq_table = {w: words.count(w) for w in word_set}
    
    # Score sentences based on frequency of their words
    sent_scores = {
        sent: sum(w_freq_table.get(w, 0) for w in word_tokenize(sent))
        for sent in sents
    }
    
    # Build summary
    multiplier = 2 * (1 - summary_size)
    
    avg = sum(sent_scores.values()) / len(sent_scores)
    summary = " ".join(sent for sent in sents if sent_scores[sent] >= avg * multiplier)
    return summary

In [101]:
text = get_text()
text

Enter a search term: Tom Cruise


('Q37079',
 'Tom Cruise',
 'Thomas Cruise Mapother IV (born July 3, 1962) is an American actor and producer. One of the world\'s highest-paid actors, he has received various accolades, including an Honorary Palme d\'Or and three Golden Globe Awards, in addition to nominations for four Academy Awards. His films have grossed over $4 billion in North America and over $11.5 billion worldwide, making him one of the highest-grossing box-office stars of all time.\nCruise began acting in the early 1980s and made his breakthrough with leading roles in the comedy film Risky Business (1983) and action film Top Gun (1986). Critical acclaim came with his roles in the dramas The Color of Money (1986), Rain Man (1988), and Born on the Fourth of July (1989). For his portrayal of Ron Kovic in the latter, he won a Golden Globe Award and received a nomination for the Academy Award for Best Actor. As a leading Hollywood star in the 1990s, he starred in several commercially successful films, including the 

In [126]:
summary = SE_summary(text[2], 0.2)
summary

'Cruise\'s performance as a motivational speaker in the drama Magnolia (1999) earned him another Golden Globe Award and a nomination for the Academy Award for Best Supporting Actor. Cruise was born on July 3, 1962, in Syracuse, New York, to electrical engineer Thomas Cruise Mapother III (1934–1984) and special education teacher Mary Lee (née Pfeiffer; 1936–2017). Their chemistry won praise among critics with The Washington Post writing, "One of the subtle achievements of both Cruise\'s and Newman\'s performances is that you feel that both of them are genuinely top-notch pool hustlers". Later that year he starred with Dustin Hoffman in Barry Levinson\'s Rain Man, which won the Academy Award for Best Film and Cruise the Kansas City Film Critics Circle Award for Best Supporting Actor. The performance earned him a Golden Globe Award for Best Actor –  Motion Picture Drama, the Chicago Film Critics Association Award for Best Actor, the People\'s Choice Award for Favorite Motion Picture Actor

In [127]:
char_len = len(text[2])
sum_char_len = len(summary)
word_len = len(nltk.word_tokenize(text[2]))
sum_word_len = len(nltk.word_tokenize(summary))
print(
    f"Character lengths: {char_len} vs {sum_char_len}", 
    f"Word lengths: {word_len} vs {sum_word_len}", 
    f"Summary char ratio: {(sum_char_len / char_len) * 100}%",
    f"Summary word ratio: {(sum_word_len / word_len) * 100}%",
    sep="\n",
)

Character lengths: 33497 vs 7436
Word lengths: 6314 vs 1359
Summary char ratio: 22.19900289578171%
Summary word ratio: 21.523598352866646%


# Clever Algorithms

In [128]:
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer

## LexRank, TextRank, Latent Semantic Analysis (LSA), Luhn's Algorithm

In [132]:
def algo_summary(text, method, summary_size=0.5):
    # Get sentence and summary lengths
    sent_length = len(nltk.sent_tokenize(text))
    summary_len = max(int(summary_size * sent_length), 1)
    print(summary_len)
    
    # Initialise summariser
    if method == "lex":
        summariser = LexRankSummarizer()
    elif method == "text":
        summariser = TextRankSummarizer()
    elif method == "lsa":
        summariser = LsaSummarizer()
    elif method == "luhn":
        summariser = LuhnSummarizer()
    
    # Initialise parser
    parser = PlaintextParser(text, Tokenizer("english"))
    
    # Create summary
    summary_sents = summariser(parser.document, summary_len)
    
    return " ".join(str(s) for s in summary_sents)

In [130]:
text = get_text()
text

Enter a search term: Tom Cruise


('Q37079',
 'Tom Cruise',
 'Thomas Cruise Mapother IV (born July 3, 1962) is an American actor and producer. One of the world\'s highest-paid actors, he has received various accolades, including an Honorary Palme d\'Or and three Golden Globe Awards, in addition to nominations for four Academy Awards. His films have grossed over $4 billion in North America and over $11.5 billion worldwide, making him one of the highest-grossing box-office stars of all time.\nCruise began acting in the early 1980s and made his breakthrough with leading roles in the comedy film Risky Business (1983) and action film Top Gun (1986). Critical acclaim came with his roles in the dramas The Color of Money (1986), Rain Man (1988), and Born on the Fourth of July (1989). For his portrayal of Ron Kovic in the latter, he won a Golden Globe Award and received a nomination for the Academy Award for Best Actor. As a leading Hollywood star in the 1990s, he starred in several commercially successful films, including the 

In [137]:
for meth in ("lex", "text", "lsa", "luhn"):
    print(f"\nMethod: {meth}", end="\n\n")
    summary = algo_summary(text[2], meth, 0.2)
    char_len = len(text[2])
    sum_char_len = len(summary)
    word_len = len(nltk.word_tokenize(text[2]))
    sum_word_len = len(nltk.word_tokenize(summary))
    print(
        f"Character lengths: {char_len} vs {sum_char_len}", 
        f"Word lengths: {word_len} vs {sum_word_len}", 
        f"Summary char ratio: {(sum_char_len / char_len) * 100}%",
        f"Summary word ratio: {(sum_word_len / word_len) * 100}%",
        sep="\n",
    )
    print(summary)
    print(f"\n{100*'='}\n")


Method: lex

50
Character lengths: 33594 vs 6455
Word lengths: 6331 vs 1261
Summary char ratio: 19.214740727510865%
Summary word ratio: 19.917864476386036%
Thomas Cruise Mapother IV (born July 3, 1962) is an American actor and producer. One of the world's highest-paid actors, he has received various accolades, including an Honorary Palme d'Or and three Golden Globe Awards, in addition to nominations for four Academy Awards. Critical acclaim came with his roles in the dramas The Color of Money (1986), Rain Man (1988), and Born on the Fourth of July (1989). For his portrayal of Ron Kovic in the latter, he won a Golden Globe Award and received a nomination for the Academy Award for Best Actor. As a leading Hollywood star in the 1990s, he starred in several commercially successful films, including the drama A Few Good Men (1992), the thriller The Firm (1993), the horror film Interview with the Vampire (1994), and the romance Jerry Maguire (1996). Cruise's performance as a motivational spe

Character lengths: 33594 vs 6854
Word lengths: 6331 vs 1231
Summary char ratio: 20.402452818955766%
Summary word ratio: 19.44400568630548%
He has played Ethan Hunt in all six of the Mission: Impossible films from 1996 to 2018. Cruise has been married to actresses Mimi Rogers, Nicole Kidman, and Katie Holmes. Cruise is an outspoken advocate for the Church of Scientology, which he credits with helping him overcome dyslexia. In the 2000s, he sparked controversy with his criticisms of psychiatry and anti-depressant drugs, his efforts to promote Scientology in Europe, and a leaked video interview of him promoting Scientology. One of his cousins, William Mapother, is also an actor who has appeared alongside Cruise in five films. He elaborated, "[My father] was the kind of person where, if something goes wrong, they kick you. He attended the new Robert Hopkins Public School for his fourth and fifth grade education. He and six other boys put on an improvised play to music called IT at the Carl

# Neural Techniques

## Facebook BART and Google T5

In [11]:
from transformers import pipeline

In [96]:
models = ["facebook/bart-large-cnn", "t5-base", "philschmid/bart-large-cnn-samsum", "pszemraj/long-t5-tglobal-base-16384-book-summary"]

In [91]:
def ann_summary(text, model_name, summary_size=0.5):
    # Get lengths of original text and summary
    word_len = len(nltk.word_tokenize(text))
    summary_len = int(summary_size * word_len)
    
    # Initialise summariser
    summariser = pipeline("summarization", model=model_name)
    
    summary = summariser(text, max_length=summary_len, min_length=summary_len, do_sample=False)
    return summary[0]['summary_text']

def ann_summary_2(text, model_name, summary_size=0.5):
    # Get lengths of original text and summary
    word_len = len(nltk.word_tokenize(text))
    summary_len = int(summary_size * word_len)
    
    # Get API url and headers
    api_url = f"https://api-inference.huggingface.co/models/{model_name}"
    headers = {"Authorization": f"Bearer {os.getenv('HUGGING_FACE_API_KEY')}"}
    
    payload = {
        "inputs": text,
        "parameters": {
            "do_sample": False,
            "max_length": 130,
            "min_length": 10
        }
    }
    
    data = json.dumps(payload)
    res = req.request("POST", api_url, headers=headers, data=data)
    
    return json.loads(res.content.decode("utf-8"))

In [81]:
text = get_text()
text

Enter a search term: Tom Cruise


('Q37079',
 'Tom Cruise',
 'Thomas Cruise Mapother IV (born July 3, 1962) is an American actor and producer. One of the world\'s highest-paid actors, he has received various accolades, including an Honorary Palme d\'Or and three Golden Globe Awards, in addition to nominations for four Academy Awards. His films have grossed over $4 billion in North America and over $11.5 billion worldwide, making him one of the highest-grossing box-office stars of all time.\nCruise began acting in the early 1980s and made his breakthrough with leading roles in the comedy film Risky Business (1983) and action film Top Gun (1986). Critical acclaim came with his roles in the dramas The Color of Money (1986), Rain Man (1988), and Born on the Fourth of July (1989). For his portrayal of Ron Kovic in the latter, he won a Golden Globe Award and received a nomination for the Academy Award for Best Actor. As a leading Hollywood star in the 1990s, he starred in several commercially successful films, including the 

In [82]:
text_chunks = chunk_text(text[2], 300)
text_chunks

[" Thomas Cruise Mapother IV (born July 3, 1962) is an American actor and producer. One of the world's highest-paid actors, he has received various accolades, including an Honorary Palme d'Or and three Golden Globe Awards, in addition to nominations for four Academy Awards. His films have grossed over $4 billion in North America and over $11.5 billion worldwide, making him one of the highest-grossing box-office stars of all time. Cruise began acting in the early 1980s and made his breakthrough with leading roles in the comedy film Risky Business (1983) and action film Top Gun (1986). Critical acclaim came with his roles in the dramas The Color of Money (1986), Rain Man (1988), and Born on the Fourth of July (1989). For his portrayal of Ron Kovic in the latter, he won a Golden Globe Award and received a nomination for the Academy Award for Best Actor. As a leading Hollywood star in the 1990s, he starred in several commercially successful films, including the drama A Few Good Men (1992),

In [77]:
for chunk in text_chunks:
    summary = ann_summary(chunk, models[0], 0.9)
    print(len(nltk.word_tokenize(chunk)), len(nltk.word_tokenize(summary)), summary, end=f"\n\n{100*'='}\n\n")

285 235 Thomas Cruise Mapother IV (born July 3, 1962) is an American actor and producer. His films have grossed over $4 billion in North America and over $11.5 billion worldwide. He is one of the highest-grossing box-office stars of all time. Cruise began acting in the early 1980s and made his breakthrough with leading roles in the comedy film Risky Business (1983) and action film Top Gun (1986) For his portrayal of Ron Kovic in Born on the Fourth of July (1989), he won a Golden Globe Award and received a nomination for the Academy Award for Best Actor. As a leading Hollywood star in the 1990s, he starred in several commercially successful films, including the drama A Few Good Men (1992), the thriller The Firm (1993), the horror film Interview with the Vampire (1994), and the romance Jerry Maguire (1996) For the latter, he won the Golden Globe and received his second Academy Award nomination. Cruise's performance as a motivational speaker in the drama Magnolia (1999) earned him another

281 232 Cruise was nominated for seven Saturn Awards between 2002 and 2009, winning once. Nine of the ten films he starred in during the decade made over $100 million at the box office. In 2005, Cruise worked again with Steven Spielberg in War of the Worlds, a loose adaptation of the H. G. Wells novel of the same name. In 2006, he returned to his role as Ethan Hunt in the third installment of the Mission Impossible film series, Mission: Impossible III. In 2007, Cruise took a rare supporting role for the second time in Lions for Lambs, which was a commercial disappointment. This was followed by an unrecognizable appearance as "Les Grossman" in the 2008 comedy Tropic Thunder with Ben Stiller, Jack Black, and Robert Downey Jr. This performance earned Cruise a Golden Globe nomination. Cruise played the central role in the historical thriller Valkyrie released on December 25, 2008, to box office success. In March 2010, Cruise completed filming the action-comedy Knight and Day, in which he r

284 230 Cruise was converted to Scientology by his first wife Mimi Rogers in 1986. New York law requires all divorce documents remain sealed, so the exact terms of the settlement are not publicly available. Cruise stated that ex-wife Katie Holmes divorced him in part to protect the couple's daughter Suri from Scientology and that Suri is no longer a practicing member of the organization. Cruise is friends with the Scientology organization's chairman David Miscavige. He has campaigned for Scientology to be afforded the status of a religion in Europe. Cruise struggled with dyslexia at an early age and has said that Scientology, specifically the L. Ron Hubbard Study Tech, helped him overcome Dyslexia. His involvement in the organization was leaked by the tabloid Star in 1990, and he publicly admitted to following Scientology in a 1992 interview with Barbara Walters. According to the book Inside Scientology: The Story of America's Most Secretive Religion by Janet Reitman, seven years after

60 45 In 2012, Cruise filed a lawsuit against In Touch and Life & Style magazines for defamation. The magazines claimed Cruise had "abandoned" his six-year-old daughter. Cruise testified that due to his work load 110 days had passed without his seeing




In [99]:
for chunk in text_chunks:
    summary = ann_summary_2(chunk, models[2], 0.1)
    print(summary)
#     print(len(nltk.word_tokenize(chunk)), len(nltk.word_tokenize(summary)), summary, end=f"\n\n{100*'='}\n\n")

[{'summary_text': "Thomas Cruise Mapother IV was born July 3, 1962. He is an American actor and producer. His films have grossed over $4 billion in North America and over $11.5 billion worldwide. He has received various accolades, including an Honorary Palme d'Or and three Golden Globe Awards."}]
[{'summary_text': 'Cruise has been married to Mimi Rogers, Nicole Kidman and Katie Holmes. He has three children, two of whom were adopted during his marriage to Kidman, and the other of whom is a biological daughter he had with Holmes. Cruise is an outspoken advocate for the Church of Scientology. He was born on July 3, 1962 in Syracuse, New York.'}]
[{'summary_text': 'Cruise spent part of his childhood in Canada. He attended 15 schools in 14 years. He became involved in drama in fourth grade under drama teacher George Steinburg. In sixth grade, his mother left him and his father and moved back to the United States.'}]
[{'summary_text': 'Tom Cruise graduated from Glen Ridge High School in New

KeyboardInterrupt: 

## Longformer

In [17]:
from transformers import LongformerTokenizer, EncoderDecoderModel
from transformers import LEDTokenizer, LEDForConditionalGeneration

In [18]:
text = get_text()
text

Enter a search term: Tom Cruise


('Q37079',
 'Tom Cruise',
 'Thomas Cruise Mapother IV (born July 3, 1962) is an American actor and producer. One of the world\'s highest-paid actors, he has received various accolades, including an Honorary Palme d\'Or and three Golden Globe Awards, in addition to nominations for four Academy Awards. His films have grossed over $4 billion in North America and over $11.5 billion worldwide, making him one of the highest-grossing box-office stars of all time.\nCruise began acting in the early 1980s and made his breakthrough with leading roles in the comedy film Risky Business (1983) and action film Top Gun (1986). Critical acclaim came with his roles in the dramas The Color of Money (1986), Rain Man (1988), and Born on the Fourth of July (1989). For his portrayal of Ron Kovic in the latter, he won a Golden Globe Award and received a nomination for the Academy Award for Best Actor. As a leading Hollywood star in the 1990s, he starred in several commercially successful films, including the 

In [19]:
def LF_summary(text, summary_size=0.5):
    # Get word length of original text
    word_len = len(nltk.word_tokenize(text))
    
    # Initialise tokeniser and model
    tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")
    model = EncoderDecoderModel.from_pretrained("patrickvonplaten/longformer2roberta-cnn_dailymail-fp16")
    
    # Chunking
    if word_len > 4096:
        chunks = []
        current_chunk = ""
        sents = nltk.sent_tokenize(text)
        for sent in sents:
            if len(nltk.word_tokenize(current_chunk + f" {sent}")) > 3000:
                chunks.append(current_chunk)
                current_chunk = ""
            else:
                current_chunk += f" {sent}"
        
        summary_chunks = []
        for chunk in chunks:
            chunk_len = len(nltk.word_tokenize(chunk))
            summary_len = int(summary_size * chunk_len)
            
            input_ids = tokenizer(chunk, return_tensors="pt").input_ids
            
            summary_ids = model.generate(input_ids, max_length=chunk_len, min_length=summary_len, num_beams=4, early_stopping=True)
            summary = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)
            print(len(nltk.word_tokenize(chunk)), summary_len, len(summary), summary, end="\n=====================================\n\n")

In [20]:
def LF_summary_2(text, summary_size=0.5):
    # Get word lengths of original text and summary
    word_len = len(nltk.word_tokenize(text))
    summary_len = min(int(summary_size * word_len), 1024)
    
    # Initialise model and tokenizer
    tokenizer = LEDTokenizer.from_pretrained("patrickvonplaten/led-large-16384-pubmed")
    model = LEDForConditionalGeneration.from_pretrained("patrickvonplaten/led-large-16384-pubmed")
    
    # Initialise input dictionary
    inputs_dict = tokenizer(text, padding="max_length", max_length=16384, return_tensors="pt", truncation=True)
    input_ids = inputs_dict.input_ids #.to("cuda")
    attention_mask = inputs_dict.attention_mask #.to("cuda")
    global_attention_mask = torch.zeros_like(attention_mask)
    global_attention_mask[:, 0] = 1

    # Create summary
    predicted_abstract_ids = model.generate(input_ids, max_length=summary_len, min_length=summary_len, attention_mask=attention_mask, global_attention_mask=global_attention_mask)
    return tokenizer.batch_decode(predicted_abstract_ids, skip_special_tokens=True)

In [22]:
LF_summary_2(text[2])

[" tom Cruise Mapother IV ( born July 3, 1962 ) is an actor and producer who is one of the highest-grossing box - office stars of all time. \n he is an outspoken advocate for the Church of Scientology, which he credits with helping him overcome dyslexia. in the 2000s, he sparked controversy with his criticisms of psychiatry and anti-depressant drugs, his efforts to promote Scientology in the united states, and a leaked video interview of him promoting Scientology. in this article \n, we review the history of his career, including his career as an actor, producer, and activist for the church of Scientology and his current career as a film producer and activist. in addition, we discuss his personal and professional controversies, and discuss his current status as a public figure and his recent legal and legal actions against the media and the media organizations. in conclusion, \n we review his career and discuss some of the controversies that have occurred in the last few years. in part

## OpenAI

In [65]:
def openai_summary(text, summary_size=0.5):
    word_len = len(nltk.word_tokenize(text))
    summary_len = int(summary_size * word_len)
    print(word_len, summary_len)
    
    openai.api_key = os.getenv("OPENAI_API_KEY")
#     prompt=f"Context: {text}\n\nCreate summary of the given context whose word count is {summary_size*100}% of that of the context."
    prompt=f"Context: {text}\n\nCreate summary of the given context."

#     res = openai.Completion.create(
#         model="text-davinci-003", 
#         prompt=prompt, 
#         temperature=0,
#         max_tokens=summary_len,
#     )
    
    res = openai.Completion.create(
        model="text-davinci-003", 
        prompt=f"Please summarize the following text in no fewer than {summary_len} words:\n\n{text}\n\nSummary:", 
        temperature=0,
        max_tokens=summary_len,
    )


    return res.choices[0].text

In [25]:
text = get_text()
text

Enter a search term: Tom Cruise


('Q37079',
 'Tom Cruise',
 'Thomas Cruise Mapother IV (born July 3, 1962) is an American actor and producer. One of the world\'s highest-paid actors, he has received various accolades, including an Honorary Palme d\'Or and three Golden Globe Awards, in addition to nominations for four Academy Awards. His films have grossed over $4 billion in North America and over $11.5 billion worldwide, making him one of the highest-grossing box-office stars of all time.\nCruise began acting in the early 1980s and made his breakthrough with leading roles in the comedy film Risky Business (1983) and action film Top Gun (1986). Critical acclaim came with his roles in the dramas The Color of Money (1986), Rain Man (1988), and Born on the Fourth of July (1989). For his portrayal of Ron Kovic in the latter, he won a Golden Globe Award and received a nomination for the Academy Award for Best Actor. As a leading Hollywood star in the 1990s, he starred in several commercially successful films, including the 

In [71]:
text_chunks = chunk_text(text[2], 500)
text_chunks

[" Thomas Cruise Mapother IV (born July 3, 1962) is an American actor and producer. One of the world's highest-paid actors, he has received various accolades, including an Honorary Palme d'Or and three Golden Globe Awards, in addition to nominations for four Academy Awards. His films have grossed over $4 billion in North America and over $11.5 billion worldwide, making him one of the highest-grossing box-office stars of all time. Cruise began acting in the early 1980s and made his breakthrough with leading roles in the comedy film Risky Business (1983) and action film Top Gun (1986). Critical acclaim came with his roles in the dramas The Color of Money (1986), Rain Man (1988), and Born on the Fourth of July (1989). For his portrayal of Ron Kovic in the latter, he won a Golden Globe Award and received a nomination for the Academy Award for Best Actor. As a leading Hollywood star in the 1990s, he starred in several commercially successful films, including the drama A Few Good Men (1992),

In [74]:
openai_summary(text_chunks[0])

478 239


" Thomas Cruise Mapother IV (born July 3, 1962) is an American actor and producer who has achieved worldwide success. He has won various awards, including an Honorary Palme d'Or and three Golden Globe Awards, and has been nominated for four Academy Awards. His films have grossed over $4 billion in North America and over $11.5 billion worldwide, making him one of the highest-grossing box-office stars of all time. Cruise began acting in the early 1980s and made his breakthrough with leading roles in the comedy film Risky Business (1983) and action film Top Gun (1986). He has since starred in several commercially successful films, including A Few Good Men (1992), The Firm (1993), Interview with the Vampire (1994), and Jerry Maguire (1996). Cruise is also known for his roles in science fiction and action films, such as Mission: Impossible (1996-2018), Vanilla Sky (2001), Minority Report (2002), The Last Samurai (2003), Collateral (2004), War of the Worlds (2005), Knight and Day (2010), Jac

In [75]:
for t in text_chunks:
    summary = openai_summary(t)
    print(len(nltk.word_tokenize(t)), len(nltk.word_tokenize(summary)), summary, end=f"\n\n{100*'='}\n\n")

478 239
478 233  Thomas Cruise Mapother IV (born July 3, 1962) is an American actor and producer who has achieved worldwide success. He has won various awards, including an Honorary Palme d'Or and three Golden Globe Awards, and has been nominated for four Academy Awards. His films have grossed over $4 billion in North America and over $11.5 billion worldwide, making him one of the highest-grossing box-office stars of all time. Cruise began acting in the early 1980s and made his breakthrough with leading roles in the comedy film Risky Business (1983) and action film Top Gun (1986). He has since starred in several commercially successful films, including A Few Good Men (1992), The Firm (1993), Interview with the Vampire (1994), and Jerry Maguire (1996). Cruise is also known for his roles in science fiction and action films, such as Mission: Impossible (1996-2018), Vanilla Sky (2001), Minority Report (2002), The Last Samurai (2003), Collateral (2004), War of the Worlds (2005), Knight and 

465 140  Tom Cruise has been a vocal advocate for Scientology, lobbying for it to be recognized as a religion in Europe and the United States. He has also been involved in various programs to introduce people to Scientology. Cruise has been criticized for his involvement with Downtown Medical, which offered 9/11 rescue workers detoxification therapy based on the works of L. Ron Hubbard. He has also been accused of attempting to convert celebrities to Scientology. Cruise has also been criticized for his comments on psychiatry, claiming that it should be outlawed and that there is no such thing as a chemical imbalance. This led to a heated argument between Cruise and Matt Lauer on NBC's Today. Medical authorities view Cruise's comments as furthering the social stigma of mental illness.


467 233
467 152  In 2006, Tom Cruise made controversial comments about psychiatry and psychoactive drugs, which led to a rift with director Steven Spielberg. In 2008, a video featuring Cruise discussing 

# DocSummariser Class

In [304]:
class DocSummariser():
    def __init__(self):
        self._corpus = dict()
    
    def get_corpus(self):
        return self._corpus

    def load_files(self, corpus):
        self._corpus = corpus
    
    def clear_files(self):
        self._corpus = dict()
    
    def _word_tokenize(self, text):
        banned = list(punctuation) + nltk.corpus.stopwords.words("english")
        return [
            w for w in nltk.word_tokenize(text) 
            if w not in banned
        ]
    
    def _chunk_text(self, text, chunk_len):
        chunks = []
        current_chunk = ""
        sents = nltk.sent_tokenize(text)

        for sent in sents:
            if len(nltk.word_tokenize(current_chunk + f" {sent}")) >= chunk_len:
                chunks.append(current_chunk)
                current_chunk = ""
            else:
                current_chunk += f" {sent}"

        chunks.append(current_chunk)

        return chunks
    
    def summarise(self, method, fnames, summary_size):
        # Build input text
        text = " ".join(self._corpus[name] for name in fnames)
        
        if method == "se":
            return self._SE_summary(text, summary_size)
        elif method in ("lexR", "texR", "lsa", "luhn"):
            return self._algo_summary(text, method, summary_size)
        elif method == "bart":
            text_chunks = self._chunk_text(text, 300)
            return " ".join(
                self._BART_summary(chunk, summary_size)
                for chunk in text_chunks
            )
        elif method == "openai":
            text_chunks = self._chunk_text(text, 500)
            return " ".join(
                self._openai_summary(chunk, summary_size)
                for chunk in text_chunks
            )
    
    def _SE_summary(self, text, summary_size=0.5):
        # Create word and sentence tokens
        words = word_tokenize(text)
        word_set = set(words) # set of all unique words in word tokens
        sents = nltk.sent_tokenize(text)

        # Initialise frequency table for word tokens
        w_freq_table = {w: words.count(w) for w in word_set}

        # Score sentences based on frequency of their words
        sent_scores = {
            sent: sum(w_freq_table.get(w, 0) for w in word_tokenize(sent))
            for sent in sents
        }

        # Build summary
        multiplier = 2 * (1 - summary_size)

        avg = sum(sent_scores.values()) / len(sent_scores)
        summary = " ".join(sent for sent in sents if sent_scores[sent] >= avg * multiplier)
        return summary
    
    def _algo_summary(self, text, method, summary_size=0.5):
        # Get sentence and summary lengths
        sent_length = len(nltk.sent_tokenize(text))
        summary_len = max(int(summary_size * sent_length), 1)

        # Initialise summariser
        if method == "lexR":
            summariser = LexRankSummarizer()
        elif method == "texR":
            summariser = TextRankSummarizer()
        elif method == "lsa":
            summariser = LsaSummarizer()
        elif method == "luhn":
            summariser = LuhnSummarizer()

        # Initialise parser
        parser = PlaintextParser(text, Tokenizer("english"))

        # Create summary
        summary_sents = summariser(parser.document, summary_len)

        return " ".join(str(s) for s in summary_sents)
    
    def _BART_summary(self, text, summary_size=0.5):
        # Get lengths of original text and summary
        word_len = len(nltk.word_tokenize(text))
        summary_len = int((summary_size * word_len) + 0.5)
        print(word_len, summary_len)

        # Get API url and headers
        api_url = "https://api-inference.huggingface.co/models/facebook/bart-large-cnn"
        headers = {"Authorization": f"Bearer {os.getenv('HUGGING_FACE_API_KEY')}"}

        payload = {
            "inputs": text,
            "parameters": {
                "do_sample": False,
#                 "max_length": max(round(summary_len + 50, -2), 2),
                "max_length": min(round(summary_len + 50, -2), word_len),
                "min_length": max(summary_len - 10, 1)
            }
        }

        data = json.dumps(payload)
        res = req.request("POST", api_url, headers=headers, data=data)
        print(res)
        text = json.loads(res.content.decode("utf-8"))[0]["summary_text"]
        
        return text
    
    def _openai_summary(self, text, summary_size=0.5):
        word_len = len(nltk.word_tokenize(text))
        summary_len = int(summary_size * word_len)
        print(word_len, summary_len)

        openai.api_key = os.getenv("OPENAI_API_KEY")
        prompt=f"Summarize the following text in no more than {summary_len} words:\n\n{text}\n\nSummary:"

        res = openai.Completion.create(
            model="text-davinci-003", 
            prompt=prompt, 
            temperature=0,
            max_tokens=round(summary_len + 50, -2),
        )

        summary = res.choices[0].text
        return summary

In [305]:
# docs = [get_text() for i in range(3)]
# doc_corpus = { doc[1] : doc[2] for doc in docs}
# doc_corpus

In [306]:
doc_corpus.keys()

dict_keys(['Tom Cruise', 'Matt Damon', 'Meryl Streep'])

In [307]:
sumObj = DocSummariser()

In [308]:
sumObj.load_files(doc_corpus)

In [309]:
sumObj.get_corpus()

{'Tom Cruise': 'Thomas Cruise Mapother IV (born July 3, 1962) is an American actor and producer. One of the world\'s highest-paid actors, he has received various accolades, including an Honorary Palme d\'Or and three Golden Globe Awards, in addition to nominations for four Academy Awards. His films have grossed over $4 billion in North America and over $11.5 billion worldwide, making him one of the highest-grossing box-office stars of all time.\nCruise began acting in the early 1980s and made his breakthrough with leading roles in the comedy film Risky Business (1983) and action film Top Gun (1986). Critical acclaim came with his roles in the dramas The Color of Money (1986), Rain Man (1988), and Born on the Fourth of July (1989). For his portrayal of Ron Kovic in the latter, he won a Golden Globe Award and received a nomination for the Academy Award for Best Actor. As a leading Hollywood star in the 1990s, he starred in several commercially successful films, including the drama A Few 

In [310]:
name = "Tom Cruise"
summary = sumObj.summarise("bart", [name], 0.5)
summary

285 143
<Response [200]>
286 143
<Response [200]>
284 142
<Response [200]>
282 141
<Response [200]>
265 133
<Response [200]>
286 143
<Response [200]>
291 146
<Response [200]>
281 141
<Response [200]>
296 148
<Response [200]>
289 145
<Response [200]>
294 147
<Response [200]>
277 139
<Response [200]>
280 140
<Response [200]>
294 147
<Response [200]>
284 142
<Response [200]>
285 143
<Response [200]>
289 145
<Response [200]>
292 146
<Response [200]>
292 146
<Response [200]>
282 141
<Response [200]>
60 30
<Response [200]>


'Thomas Cruise Mapother IV (born July 3, 1962) is an American actor and producer. His films have grossed over $4 billion in North America and over $11.5 billion worldwide. He is one of the highest-grossing box-office stars of all time. Cruise began acting in the early 1980s and made his breakthrough with leading roles in the comedy film Risky Business (1983) and action film Top Gun (1986) For his portrayal of Ron Kovic in Born on the Fourth of July (1989), he won a Golden Globe Award and received a nomination for the Academy Award for Best Actor. In the 1990s, he starred in several commercially successful films, including the drama A Few Good Men (1992), the thriller The Firm (1993), the horror film Interview with the Vampire (1994), and the romance Jerry Maguire (1996) Cruise has been married to actresses Mimi Rogers, Nicole Kidman, and Katie Holmes. He has three children, two of whom were adopted during his marriage to Kidman and the other of whom is a biological daughter he had with

In [311]:
char_len = len(sumObj.get_corpus()[name])
sum_char_len = len(summary)
word_len = len(nltk.word_tokenize(sumObj.get_corpus()[name]))
sum_word_len = len(nltk.word_tokenize(summary))
print(
    f"Character lengths: {char_len} vs {sum_char_len}", 
    f"Word lengths: {word_len} vs {sum_word_len}", 
    f"Summary char ratio: {(sum_char_len / char_len) * 100}%",
    f"Summary word ratio: {(sum_word_len / word_len) * 100}%",
    sep="\n",
)
print(f"\n{100*'='}\n")

Character lengths: 33594 vs 14304
Word lengths: 6331 vs 2732
Summary char ratio: 42.57903196999464%
Summary word ratio: 43.15274048333597%




In [312]:
sumObj2 = DocSummariser()
sumObj2.load_files({"TheOne": "Just the one sentence."})
sumObj2.get_corpus()

{'TheOne': 'Just the one sentence.'}

In [314]:
sumObj2.summarise("bart", ["TheOne"], 0.5)

5 3
<Response [200]>


'Just the'

In [315]:
round(5 + 50, -2)

100